In [13]:
import torch
from torch import Tensor
import numpy as np
from torch import nn
import dlc_bci as bci
from torch.autograd import Variable
from torch import optim
from torch.nn import functional as F
import matplotlib.pyplot as plt
from multiprocessing import Pool
%matplotlib inline

from sklearn import model_selection

In [14]:
train_input, train_target = bci.load(root = './data_bci', one_khz=True)
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
X = train_input.numpy()
y = train_target.numpy()
kfolds = model_selection.KFold(n_splits=10, random_state=1234, shuffle=True)

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])


In [15]:
test_input , test_target = bci.load(root = './data_bci', train = False, one_khz=True)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [17]:
# put this inside the train to avoid data snooping
mu, std = train_input.mean(0), train_input.std(0)
train_input.sub_(mu).div_(std)
test_input.sub_(mu).div_(std)
print("Normalization is done!")

Normalization is done!


In [18]:
import numpy as np
import pprint
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable

from utils import compute_nb_errors


def train_model_full(network_model,
                     param,
                     X, y,
                     mini_batch_size,
                     kfolds,
                     nb_epochs,
                     lambdda=0.01,
                     lr=0.001,
                     verbose=False):
    acc_train_kfold = []
    loss_train_kfold = []
    acc_val_kfold = []
    loss_val_kfold = []

    for d, (train_index, val_index) in enumerate(kfolds.split(X)):
        if verbose:
            print('\nFold {}'.format(d))
            print('----------------------------------------------------------------')

        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        X_train = Variable(torch.from_numpy(X_train))
        X_val = Variable(torch.from_numpy(X_val))
        y_train = Variable(torch.from_numpy(y_train))
        y_val = Variable(torch.from_numpy(y_val))

        model = network_model(**param)

        loss_train, loss_val, acc_train, acc_val = train_model(model,
                                                               X_train, y_train,
                                                               X_val, y_val,
                                                               mini_batch_size,
                                                               nb_epochs,
                                                               lambdda, lr,
                                                               verbose)
        acc_train_kfold.append(acc_train)
        loss_train_kfold.append(loss_train)
        acc_val_kfold.append(acc_val)
        loss_val_kfold.append(loss_val)

    acc_train_kfold = np.mean(np.array(acc_train_kfold), axis=0)
    acc_val_kfold = np.mean(np.array(acc_val_kfold), axis=0)

    loss_train_kfold = np.mean(np.array(loss_train_kfold), axis=0)
    loss_val_kfold = np.mean(np.array(loss_val_kfold), axis=0)

    pp = pprint.PrettyPrinter(indent=4)
    print('{} epochs done for `{}` with the parameters:'.format(nb_epochs, network_model.__name__))
    pp.pprint(param)
    print('\n   Loss: train ~ {} Acc train ~ {} \n   Loss: val ~ {} / Acc val ~ {}'
          .format(round(loss_train_kfold[-1], 3),
                  round(acc_train_kfold[-1], 3),
                  round(loss_val_kfold[-1], 3),
                  round(acc_val_kfold[-1], 3)))
    print('----------------------------------------------------------------')
    return loss_train_kfold, loss_val_kfold, acc_train_kfold, acc_val_kfold


def train_model(model, X_train, y_train, X_val, y_val, mini_batch_size, nb_epochs, lambdda=0.01, lr=0.001,
                verbose=False):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr)

    acc_train = []
    acc_val = []
    loss_train = []
    loss_val = []

    for e in range(0, nb_epochs):

        model.train(True)
        for b in list(range(0, X_train.size(0), mini_batch_size)):
            if b + mini_batch_size <= X_train.size(0):
                output = model(X_train.narrow(0, b, mini_batch_size))
                loss = criterion(output, y_train.narrow(0, b, mini_batch_size))
            else:
                output = model(X_train.narrow(0, b, X_train.size(0) - b))
                loss = criterion(output, y_train.narrow(0, b, X_train.size(0) - b))

            for p in model.parameters():
                loss += lambdda * p.pow(2).sum()
            model.zero_grad()
            loss.backward()
            optimizer.step()

            model.train(False)
            output_train = model(X_train)
            output_val = model(X_val)

        acc_val.append(1 - compute_nb_errors(model, X_val, y_val, mini_batch_size=mini_batch_size) / X_val.size(0))
        acc_train.append(
            1 - compute_nb_errors(model, X_train, y_train, mini_batch_size=mini_batch_size) / X_train.size(0))
        loss_train.append(criterion(output_train, y_train).data[0])
        loss_val.append(criterion(output_val, y_val).data[0])

        if verbose and (e-1) % 100 == 0:
            print('Epoch {}, accuracy in validation: {} / train {}'.format(e, round(acc_val[-1], 3), round(acc_train[-1], 3)))

    return loss_train, loss_val, acc_train, acc_val


def train_parallel(param):
    train_model_full(**param)


In [19]:
import torch

def compute_nb_errors(model, data_input, data_target, mini_batch_size):
    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        if b + mini_batch_size <= data_input.size(0):
            output = model(data_input.narrow(0, b, mini_batch_size))
            _, predicted_classes = torch.max(output.data, 1)
            for k in range(0, mini_batch_size):
                if data_target.data[b + k] != predicted_classes[k]:
                    nb_data_errors = nb_data_errors + 1
        else:
            output = model(data_input.narrow(0, b, data_input.size(0) - b))
            _, predicted_classes = torch.max(output.data, 1)
            for k in range(0, data_input.size(0) - b):
                if data_target.data[b + k] != predicted_classes[k]:
                    nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [20]:
MINI_BATCH_SIZE = 40
N_EPOCHS = 3
N_FOLDS = 10
kfolds = model_selection.KFold(n_splits=N_FOLDS, random_state=1234, shuffle=True)

In [21]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, dropout):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size,
        self.hidden_size = hidden_size
        self.dropout = dropout

        self.rnn = nn.LSTM(         
            input_size=self.input_size,
            hidden_size=self.hidden_size,         
            num_layers=1,           
            batch_first=True,  
            dropout=self.dropout
        )

        self.out = nn.Linear(self.hidden_size, 2)

    def forward(self, x):
        out, (h_n, h_c) = self.rnn(x, None) # zero initial hidden state
        out = self.out(out[:, -1, :])
        return out


In [24]:
costs, costs_val, acc, acc_val = train_model_full(RNN,
                                                              {'input_size': 500,
                                                               'output_size': 1,
                                                               'hidden_size': 128,
                                                               'dropout': 0.1},
                                                              X,
                                                              y,
                                                              MINI_BATCH_SIZE,
                                                              kfolds,
                                                              502,
                                                              lambdda=0.0375, 
                                                  verbose=True)


Fold 0
----------------------------------------------------------------
Epoch 1, accuracy in validation: 0.531 / train 0.644
Epoch 101, accuracy in validation: 0.25 / train 0.648
Epoch 201, accuracy in validation: 0.281 / train 0.69
Epoch 301, accuracy in validation: 0.344 / train 0.68
Epoch 401, accuracy in validation: 0.406 / train 0.722
Epoch 501, accuracy in validation: 0.344 / train 0.757

Fold 1
----------------------------------------------------------------
Epoch 1, accuracy in validation: 0.594 / train 0.532
Epoch 101, accuracy in validation: 0.625 / train 0.697
Epoch 201, accuracy in validation: 0.719 / train 0.676
Epoch 301, accuracy in validation: 0.594 / train 0.778
Epoch 401, accuracy in validation: 0.562 / train 0.82
Epoch 501, accuracy in validation: 0.594 / train 0.683

Fold 2
----------------------------------------------------------------
Epoch 1, accuracy in validation: 0.469 / train 0.602
Epoch 101, accuracy in validation: 0.375 / train 0.701
Epoch 201, accuracy i

KeyboardInterrupt: 